In [1]:
import pandas as pd
movies=pd.read_csv(r"C:\project news\movies.csv")
print("movies dataset:")
print(movies.head())
rating=pd.read_csv(r"C:\project news\ratings.csv")
print("\n rating dateset:")
print(rating.head())

movies dataset:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

 rating dateset:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [2]:
print("Missing values int movies dataset:\n",movies.isnull().sum())
print("\nMissing values in ratings dataset:\n",rating.isnull().sum())
ratings = rating.drop(columns=['timestamp'])
movie_rating = pd.merge(ratings, movies, on="movieId")
print("\nMerged Dataset:")
print(movie_rating.head())
movie_count = movie_rating['title'].value_counts()
print("\nTop 10 Movies with most rating:\n",movie_count.head(10))


Missing values int movies dataset:
 movieId    0
title      0
genres     0
dtype: int64

Missing values in ratings dataset:
 userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

Merged Dataset:
   userId  movieId  rating                        title  \
0       1        1     4.0             Toy Story (1995)   
1       1        3     4.0      Grumpier Old Men (1995)   
2       1        6     4.0                  Heat (1995)   
3       1       47     5.0  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  

Top 10 Movies with most rating:
 title
Forrest Gump (1994)                          329
Shawshank Redemption, The (1994)             31

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
tfidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(movies['genres'].fillna(''))
cosine_sim=cosine_similarity(tfidf_matrix,tfidf_matrix)


In [5]:
def recommend_movies(title,movies_df,cosine_sim):
    
    movies_df['title'] = movies_df['title'].str.strip().str.lower()
    title = title.strip().lower()
    if title not in movies_df['title'].values:
        print(f"Error: '{title}' not found in dataset. Try another movie.")
        return []
    indices= pd.Series(movies_df.index,index=movies_df['title']).drop_duplicates()
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[1:11]
    movies_indices=[i[0] for i in sim_scores]
    return movies_df['title'].iloc[movies_indices]
print("Recommended Movies for 'toy Story (1995)':\n", recommend_movies('toy Story (1995)', movies, cosine_sim))

Recommended Movies for 'toy Story (1995)':
 1706                                          antz (1998)
2355                                   toy story 2 (1999)
2809       adventures of rocky and bullwinkle, the (2000)
3000                     emperor's new groove, the (2000)
3568                                monsters, inc. (2001)
6194                                     wild, the (2006)
6486                               shrek the third (2007)
6948                       tale of despereaux, the (2008)
7760    asterix and the vikings (astérix et les viking...
8219                                         turbo (2013)
Name: title, dtype: object


In [6]:
print("Recommended Movies for ' Tom and Huck (1995)':\n", recommend_movies(' Tom and Huck (1995)', movies, cosine_sim))


Recommended Movies for ' Tom and Huck (1995)':
 119                  amazing panda adventure, the (1995)
131                                        casper (1995)
204    far from home: the adventures of yellow dog (1...
421                                        lassie (1994)
521      homeward bound ii: lost in san francisco (1996)
580                                       flipper (1996)
635                                        alaska (1996)
642                  adventures of pinocchio, the (1996)
751                                 fly away home (1996)
775                         swiss family robinson (1960)
Name: title, dtype: object


In [7]:
print(movies['title'].head(15)) 


0                       toy story (1995)
1                         jumanji (1995)
2                grumpier old men (1995)
3               waiting to exhale (1995)
4     father of the bride part ii (1995)
5                            heat (1995)
6                         sabrina (1995)
7                    tom and huck (1995)
8                    sudden death (1995)
9                       goldeneye (1995)
10        american president, the (1995)
11    dracula: dead and loving it (1995)
12                          balto (1995)
13                          nixon (1995)
14               cutthroat island (1995)
Name: title, dtype: object


In [8]:
print("Recommended Movies for ' bahubali(2015)':\n", recommend_movies(' bahubali(2015)', movies, cosine_sim))


Error: 'bahubali(2015)' not found in dataset. Try another movie.
Recommended Movies for ' bahubali(2015)':
 []


In [9]:
print("Recommended Movies for ' captain america':\n", recommend_movies(' captain america', movies, cosine_sim))


Error: 'captain america' not found in dataset. Try another movie.
Recommended Movies for ' captain america':
 []


In [10]:
print("Recommended Movies for ' nixon (1995)':\n", recommend_movies(' nixon (1995)', movies, cosine_sim))


Recommended Movies for ' nixon (1995)':
 25                       othello (1995)
30               dangerous minds (1995)
36      cry, the beloved country (1995)
39                   restoration (1995)
50                       georgia (1995)
51         home for the holidays (1995)
55            mr. holland's opus (1995)
105     boys of st. vincent, the (1992)
120      basketball diaries, the (1995)
121    awfully big adventure, an (1995)
Name: title, dtype: object


In [11]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
print("libaries are okay")

libaries are okay


In [12]:
ratings_matrix=ratings.pivot(index='userId',columns='movieId',values='rating')
rating_matrix=ratings_matrix.fillna(0)
print("rating matrix shape:",rating_matrix.shape)

rating matrix shape: (610, 9724)


In [13]:
ratings_df = pd.DataFrame(ratings_matrix)
ratings_matrix = ratings_df.fillna(ratings_df.mean()).values
user_mean=np.mean(ratings_matrix,axis=1).reshape(-1,1)
ratings_matrix_normalized=ratings_matrix-user_mean

In [14]:
svd= TruncatedSVD(n_components=500,random_state=42)
matrix_svd =svd.fit_transform(ratings_matrix)
print("SVD Transformed shape:",matrix_svd.shape)

SVD Transformed shape: (610, 500)


In [15]:
print("Explained varience ratio:",svd.explained_variance_ratio_.sum())

Explained varience ratio: 0.991733017766278


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
item_similarity=cosine_similarity(ratings_matrix_normalized.T).T
user_similarity=cosine_similarity(ratings_matrix_normalized)

In [17]:
def predict_ratings(similarity_matrix,ratings_matrix):
    sum_of_similarities=np.array(np.abs(similarity_matrix).sum(axis=1)).reshape(-1,1)
    predicted_ratings=similarity_matrix.dot(ratings_matrix)/sum_of_similarities
    return predicted_ratings

In [18]:
user_based_ratings = predict_ratings(user_similarity, ratings_matrix_normalized)
print("User-Based CF Predictions:\n", user_based_ratings)
item_based_ratings = predict_ratings(item_similarity, ratings_matrix_normalized.T).T
print("Item-Based CF Predictions:\n", item_based_ratings)


User-Based CF Predictions:
 [[ 0.6593288   0.1703368  -0.00228421 ...  0.23748994  0.23748994
   0.73748994]
 [ 0.6592047   0.17023198 -0.00244189 ...  0.23749801  0.23749801
   0.73749801]
 [ 0.65918251  0.17021049 -0.00245776 ...  0.23749983  0.23749983
   0.73749983]
 ...
 [ 0.65873616  0.16988974 -0.00282341 ...  0.23750486  0.23750486
   0.73750486]
 [ 0.65916282  0.17020958 -0.00245318 ...  0.23749882  0.23749882
   0.73749882]
 [ 0.65953931  0.17040773 -0.00236451 ...  0.23748302  0.23748302
   0.73748302]]
Item-Based CF Predictions:
 [[ 0.72794357  0.72533704 -0.26463017 ...  0.72859878  0.72859878
   0.72854123]
 [ 0.72148287  0.71804838 -0.31565898 ...  0.72249377  0.72249377
   0.72239403]
 [ 0.71947632  0.71586279 -0.32223234 ...  0.72060305  0.72060305
   0.72049092]
 ...
 [ 0.71903203  0.71312103 -0.43919538 ...  0.72139836  0.72139836
   0.72126656]
 [ 0.72050511  0.71695258 -0.31807869 ...  0.72159771  0.72159771
   0.72149467]
 [ 0.73439438  0.73164851 -0.3006821  ... 

In [19]:
alpha=0.5
beta=0.5
hybrid_ratings = alpha * user_based_ratings + beta *item_based_ratings
print("Hybrid Model Prediction:\n",hybrid_ratings)

Hybrid Model Prediction:
 [[ 0.69363619  0.44783692 -0.13345719 ...  0.48304436  0.48304436
   0.73301559]
 [ 0.69034378  0.44414018 -0.15905044 ...  0.47999589  0.47999589
   0.72994602]
 [ 0.68932941  0.44303664 -0.16234505 ...  0.47905144  0.47905144
   0.72899538]
 ...
 [ 0.6888841   0.44150539 -0.22100939 ...  0.47945161  0.47945161
   0.72938571]
 [ 0.68983397  0.44358108 -0.16026594 ...  0.47954827  0.47954827
   0.72949675]
 [ 0.69696684  0.45102812 -0.1515233  ...  0.48593735  0.48593735
   0.73592585]]


In [20]:
def recommend_movies_hybrid(user_id, ratings_matrix, hybrid_ratings, movies, num_recommendations=5):
    user_predictions = hybrid_ratings[user_id]
    unrated_movies = np.where(ratings_matrix[user_id] == 0)[0]
    recommended_movie_indices = unrated_movies[np.argsort(user_predictions[unrated_movies])[::-1]]
    return movies.iloc[recommended_movie_indices[:num_recommendations]]


In [21]:
user_id=6
print(user_id in range(ratings_matrix.shape[0]))
print(hybrid_ratings[user_id])


True
[ 0.69177715  0.44549703 -0.15973164 ...  0.48135451  0.48135451
  0.73130028]


In [22]:
print(movies.head())
print(ratings['userId'].unique()[:10])


   movieId                               title  \
0        1                    toy story (1995)   
1        2                      jumanji (1995)   
2        3             grumpier old men (1995)   
3        4            waiting to exhale (1995)   
4        5  father of the bride part ii (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
[ 1  2  3  4  5  6  7  8  9 10]


In [23]:
recommended_movie_indices = hybrid_ratings[user_id].argsort()[::-1][:10]
print("Recommended movie indices:", recommended_movie_indices)
print(movies.iloc[recommended_movie_indices])


Recommended movie indices: [7117 8994 8967 8968 8940 8943 1310 8921 4759 8964]
      movieId                                     title  \
7117    70994                       halloween ii (2009)   
8994   139642                           southpaw (2015)   
8967   137337                                amy (2015)   
8968   137345                  that demon within (2014)   
8940   136449             ghost in the shell 2.0 (2008)   
8943   136503  tom and jerry: shiver me whiskers (2006)   
1310     1757   fallen angels (duo luo tian shi) (1995)   
8921   135815           the magnificent ruffians (1979)   
4759     7085                 send me no flowers (1964)   
8964   136912                   god loves caviar (2012)   

                         genres  
7117            Horror|Thriller  
8994               Action|Drama  
8967                Documentary  
8968             Crime|Thriller  
8940    Action|Animation|Sci-Fi  
8943  Animation|Children|Comedy  
1310              Drama|Romance  